In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

import pandas as pd
sc = spark.sparkContext

In [3]:
import os

In [4]:
titanic_train_csv = os.path.join('input', 'titanic', 'train.csv')

In [5]:
df = spark.read.load(titanic_train_csv, format='csv', header=True, inferSchema=True)

In [6]:
df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [7]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In R's dplyr package, Hadley Wickham defined the 5 basic verbs - select, filter, mutate, summarize, and arrange. Here are the equivalents for Spark Dataframes.

In [8]:
df.select(['PassengerId', 'Survived']).show(5)

+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|          1|       0|
|          2|       1|
|          3|       1|
|          4|       1|
|          5|       0|
+-----------+--------+
only showing top 5 rows



In [9]:
df.select(['Name', 'Sex', 'Survived']).filter(df.Sex == 'female').show(5)

+--------------------+------+--------+
|                Name|   Sex|Survived|
+--------------------+------+--------+
|Cumings, Mrs. Joh...|female|       1|
|Heikkinen, Miss. ...|female|       1|
|Futrelle, Mrs. Ja...|female|       1|
|Johnson, Mrs. Osc...|female|       1|
|Nasser, Mrs. Nich...|female|       1|
+--------------------+------+--------+
only showing top 5 rows



In [10]:
#Filter also works with SQL expressions
df.select(['Name', 'Sex', 'Survived']).filter("Sex='female' AND Survived=0").show(5)

+--------------------+------+--------+
|                Name|   Sex|Survived|
+--------------------+------+--------+
|Vestrom, Miss. Hu...|female|       0|
|Vander Planke, Mr...|female|       0|
|Palsson, Miss. To...|female|       0|
|Vander Planke, Mi...|female|       0|
|Ahlin, Mrs. Johan...|female|       0|
+--------------------+------+--------+
only showing top 5 rows



In [11]:
#Mutate, i.e. adding new columns, seems to only be doable by chaining .withColumn()
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def getLastName(fullName):
    return fullName.split(',')[0]

getLastName_udf = udf(lambda x: getLastName(x), StringType())

df.select(['Name', 'Sex']).withColumn('Last Name', getLastName_udf(df.Name)).show(5)

+--------------------+------+---------+
|                Name|   Sex|Last Name|
+--------------------+------+---------+
|Braund, Mr. Owen ...|  male|   Braund|
|Cumings, Mrs. Joh...|female|  Cumings|
|Heikkinen, Miss. ...|female|Heikkinen|
|Futrelle, Mrs. Ja...|female| Futrelle|
|Allen, Mr. Willia...|  male|    Allen|
+--------------------+------+---------+
only showing top 5 rows



In [12]:
#Summarize
gdf = df.groupby('Sex', 'Pclass').agg({
    '*': 'count',
    'Age': 'avg',
    'Fare': 'sum'
}).toDF('Sex', 'Pclass', 'count', 'average_age', 'total_fare')
gdf.show()

+------+------+-----+------------------+------------------+
|   Sex|Pclass|count|       average_age|        total_fare|
+------+------+-----+------------------+------------------+
|  male|     3|  347|26.507588932806325| 4393.586500000005|
|female|     3|  144|             21.75|2321.1086000000005|
|female|     1|   94| 34.61176470588235| 9975.824999999999|
|female|     2|   76|28.722972972972972|         1669.7292|
|  male|     2|  108| 30.74070707070707|         2132.1125|
|  male|     1|  122| 41.28138613861386| 8201.587500000001|
+------+------+-----+------------------+------------------+



In [13]:
#Arrange
gdf.orderBy(['Sex', 'Pclass'], ascending=[False, True]).show()

+------+------+-----+------------------+------------------+
|   Sex|Pclass|count|       average_age|        total_fare|
+------+------+-----+------------------+------------------+
|  male|     1|  122| 41.28138613861386| 8201.587500000001|
|  male|     2|  108| 30.74070707070707|         2132.1125|
|  male|     3|  347|26.507588932806325| 4393.586500000005|
|female|     1|   94| 34.61176470588235| 9975.824999999999|
|female|     2|   76|28.722972972972972|         1669.7292|
|female|     3|  144|             21.75|2321.1086000000005|
+------+------+-----+------------------+------------------+



In [14]:
#Joins
df1 = df.select(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age'])
df2 = df.select(['PassengerId', 'Survived'])
df1.show(5)
df2.show(5)

+-----------+------+--------------------+------+----+
|PassengerId|Pclass|                Name|   Sex| Age|
+-----------+------+--------------------+------+----+
|          1|     3|Braund, Mr. Owen ...|  male|22.0|
|          2|     1|Cumings, Mrs. Joh...|female|38.0|
|          3|     3|Heikkinen, Miss. ...|female|26.0|
|          4|     1|Futrelle, Mrs. Ja...|female|35.0|
|          5|     3|Allen, Mr. Willia...|  male|35.0|
+-----------+------+--------------------+------+----+
only showing top 5 rows

+-----------+--------+
|PassengerId|Survived|
+-----------+--------+
|          1|       0|
|          2|       1|
|          3|       1|
|          4|       1|
|          5|       0|
+-----------+--------+
only showing top 5 rows



In [15]:
df1.join(df2, ['PassengerId']).show(5)

+-----------+------+--------------------+------+----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|Survived|
+-----------+------+--------------------+------+----+--------+
|          1|     3|Braund, Mr. Owen ...|  male|22.0|       0|
|          2|     1|Cumings, Mrs. Joh...|female|38.0|       1|
|          3|     3|Heikkinen, Miss. ...|female|26.0|       1|
|          4|     1|Futrelle, Mrs. Ja...|female|35.0|       1|
|          5|     3|Allen, Mr. Willia...|  male|35.0|       0|
+-----------+------+--------------------+------+----+--------+
only showing top 5 rows



In [16]:
%%time
#Non-equi joins, i.e. joining not by column name, but by values in column
#Slow due to skewed data, but at least it can be done in Spark, unlike Hive

"""
In this case, because the dataframes I am joining have the same origin, it's necessary to give them
aliases so that Spark doesn't think that I'm performing a trivial crossjoin.
Ordinarily, between dataframes of separate lineage, aliases are unnecessary,
and the following syntax would suffice:
"""

#df4.join(df5, df4.PassengerId <= df5.PassengerId).show()b

from pyspark.sql.functions import col

df3 = df.select(['PassengerId', 'Pclass','Name', 'Sex', 'Survived']).filter("PassengerId<=5")
df4 = df3.select(['PassengerId', 'Pclass', 'Name', 'Sex']).alias('dataframe1')
df5 = df3.select(['PassengerId', 'Survived']).alias('dataframe2')

df4.join(df5, on=col('dataframe1.PassengerId') <= col('dataframe2.PassengerId')).show()

+-----------+------+--------------------+------+-----------+--------+
|PassengerId|Pclass|                Name|   Sex|PassengerId|Survived|
+-----------+------+--------------------+------+-----------+--------+
|          1|     3|Braund, Mr. Owen ...|  male|          1|       0|
|          1|     3|Braund, Mr. Owen ...|  male|          2|       1|
|          1|     3|Braund, Mr. Owen ...|  male|          3|       1|
|          1|     3|Braund, Mr. Owen ...|  male|          4|       1|
|          1|     3|Braund, Mr. Owen ...|  male|          5|       0|
|          2|     1|Cumings, Mrs. Joh...|female|          2|       1|
|          2|     1|Cumings, Mrs. Joh...|female|          3|       1|
|          2|     1|Cumings, Mrs. Joh...|female|          4|       1|
|          2|     1|Cumings, Mrs. Joh...|female|          5|       0|
|          3|     3|Heikkinen, Miss. ...|female|          3|       1|
|          3|     3|Heikkinen, Miss. ...|female|          4|       1|
|          3|     3|

In [17]:
#Union, i.e. append dataframes. Columns must be shared.
df3.union(df3).show()

"""
In loops we sometimes chain union() objects, e.g. to use as a stack, which may cause performance issues
repartition() and checkpoint() may help in solving this problem
"""

+-----------+------+--------------------+------+--------+
|PassengerId|Pclass|                Name|   Sex|Survived|
+-----------+------+--------------------+------+--------+
|          1|     3|Braund, Mr. Owen ...|  male|       0|
|          2|     1|Cumings, Mrs. Joh...|female|       1|
|          3|     3|Heikkinen, Miss. ...|female|       1|
|          4|     1|Futrelle, Mrs. Ja...|female|       1|
|          5|     3|Allen, Mr. Willia...|  male|       0|
|          1|     3|Braund, Mr. Owen ...|  male|       0|
|          2|     1|Cumings, Mrs. Joh...|female|       1|
|          3|     3|Heikkinen, Miss. ...|female|       1|
|          4|     1|Futrelle, Mrs. Ja...|female|       1|
|          5|     3|Allen, Mr. Willia...|  male|       0|
+-----------+------+--------------------+------+--------+



'\nIn loops we sometimes chain union() objects, e.g. to use as a stack, which may cause performance issues\nrepartition() and checkpoint() may help in solving this problem\n'

In [19]:
#Write to file
df2.write.csv(os.path.join('output', 'titanic', 'train_targets'))

In [20]:
"""
Almost everything done above can be done by writing Hive SQL queries in spark.sql()
The queries would act on tables, not dataframes, so to illustrate we'll convert the dataframes to 
temp tables
"""

df1.createOrReplaceTempView('df1_temp')
df2.createOrReplaceTempView('df2_temp')

query='''
    select
        a.PassengerId,
        a.Pclass,
        a.Name,
        a.Sex,
        a.Age,
        b.Survived
    from df1_temp a
    join df2_temp b
        on a.PassengerId = b.PassengerId
'''
dfj = spark.sql(query)
dfj.show()

+-----------+------+--------------------+------+----+--------+
|PassengerId|Pclass|                Name|   Sex| Age|Survived|
+-----------+------+--------------------+------+----+--------+
|          1|     3|Braund, Mr. Owen ...|  male|22.0|       0|
|          2|     1|Cumings, Mrs. Joh...|female|38.0|       1|
|          3|     3|Heikkinen, Miss. ...|female|26.0|       1|
|          4|     1|Futrelle, Mrs. Ja...|female|35.0|       1|
|          5|     3|Allen, Mr. Willia...|  male|35.0|       0|
|          6|     3|    Moran, Mr. James|  male|null|       0|
|          7|     1|McCarthy, Mr. Tim...|  male|54.0|       0|
|          8|     3|Palsson, Master. ...|  male| 2.0|       0|
|          9|     3|Johnson, Mrs. Osc...|female|27.0|       1|
|         10|     2|Nasser, Mrs. Nich...|female|14.0|       1|
|         11|     3|Sandstrom, Miss. ...|female| 4.0|       1|
|         12|     1|Bonnell, Miss. El...|female|58.0|       1|
|         13|     3|Saundercock, Mr. ...|  male|20.0|  

In [21]:
dfj.explain()

== Physical Plan ==
*(2) Project [PassengerId#10, Pclass#12, Name#13, Sex#14, Age#15, Survived#424]
+- *(2) BroadcastHashJoin [PassengerId#10], [PassengerId#423], Inner, BuildRight
   :- *(2) Project [PassengerId#10, Pclass#12, Name#13, Sex#14, Age#15]
   :  +- *(2) Filter isnotnull(PassengerId#10)
   :     +- *(2) FileScan csv [PassengerId#10,Pclass#12,Name#13,Sex#14,Age#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/friedemann.ang/Documents/repos/pyspark-learnbook/input/titanic/t..., PartitionFilters: [], PushedFilters: [IsNotNull(PassengerId)], ReadSchema: struct<PassengerId:int,Pclass:int,Name:string,Sex:string,Age:double>
   +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
      +- *(1) Project [PassengerId#423, Survived#424]
         +- *(1) Filter isnotnull(PassengerId#423)
            +- *(1) FileScan csv [PassengerId#423,Survived#424] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/

There are two types of operations in Spark: transformations and actions. 

.explain() lays out the plan to transform some input to obtain some further state, but it isn't executed until some action is called, e.g. show(), count(), saveAsTable() etc.

A good execution plan means good performance, so examining the plan using explain() is a good way to tune the performance of Spark jobs.

In [22]:
"""
Caching

Cache a dataframe when it is used multiple times in a script.
This can reduce the number of times an execution plan is carried out to obtain the dataframe, 
and so improve performance.
"""
#cache() only works after at least one action has been called on a dataframe
df1.cache()
#to see if a dataframe is cachhed, check the storageLevel property
df1.storageLevel

StorageLevel(True, True, False, True, 1)

In [23]:
#to uncache a dataframe, use unpersist()
df1.unpersist()
df1.storageLevel

StorageLevel(False, False, False, False, 1)

In [24]:
"""
Checkpointing

checkpoint()  truncates an execution plan and caches the checkpointed dataframe to a temporary
location on the disk. 

It's recommended that you cache before checkpointing so that Spark doesn't have to read the dataframe
from disk again after it's checkpointed.
"""

#To use checkpoint(), it's necessary to specify the temporary file location to save the dataframe to, 
#by accessing the sparkContext object from SparkSession

sc.setCheckpointDir('/checkpointdir') #save to c:/checkpointdir

In [25]:
dfc = df1.join(df1, ['PassengerId'])
dfc.join(df1, ['PassengerId']).explain()

== Physical Plan ==
*(3) Project [PassengerId#10, Pclass#12, Name#13, Sex#14, Age#15, Pclass#500, Name#501, Sex#502, Age#503, Pclass#521, Name#522, Sex#523, Age#524]
+- *(3) BroadcastHashJoin [PassengerId#10], [PassengerId#519], Inner, BuildRight
   :- *(3) Project [PassengerId#10, Pclass#12, Name#13, Sex#14, Age#15, Pclass#500, Name#501, Sex#502, Age#503]
   :  +- *(3) BroadcastHashJoin [PassengerId#10], [PassengerId#498], Inner, BuildRight
   :     :- *(3) Project [PassengerId#10, Pclass#12, Name#13, Sex#14, Age#15]
   :     :  +- *(3) Filter isnotnull(PassengerId#10)
   :     :     +- *(3) FileScan csv [PassengerId#10,Pclass#12,Name#13,Sex#14,Age#15] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/friedemann.ang/Documents/repos/pyspark-learnbook/input/titanic/t..., PartitionFilters: [], PushedFilters: [IsNotNull(PassengerId)], ReadSchema: struct<PassengerId:int,Pclass:int,Name:string,Sex:string,Age:double>
   :     +- BroadcastExchange HashedRelationBroadcast

In [26]:
#Checkpointing after the first join would truncate the plan
dfc2 = df1.join(df1, ['PassengerId']).checkpoint()
dfc2.join(df1, ['PassengerId']).explain()

== Physical Plan ==
*(2) Project [PassengerId#10, Pclass#12, Name#13, Sex#14, Age#15, Pclass#546, Name#547, Sex#548, Age#549, Pclass#587, Name#588, Sex#589, Age#590]
+- *(2) BroadcastHashJoin [PassengerId#10], [PassengerId#585], Inner, BuildRight
   :- *(2) Filter isnotnull(PassengerId#10)
   :  +- Scan ExistingRDD[PassengerId#10,Pclass#12,Name#13,Sex#14,Age#15,Pclass#546,Name#547,Sex#548,Age#549]
   +- BroadcastExchange HashedRelationBroadcastMode(List(cast(input[0, int, true] as bigint)))
      +- *(1) Project [PassengerId#585, Pclass#587, Name#588, Sex#589, Age#590]
         +- *(1) Filter isnotnull(PassengerId#585)
            +- *(1) FileScan csv [PassengerId#585,Pclass#587,Name#588,Sex#589,Age#590] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/C:/Users/friedemann.ang/Documents/repos/pyspark-learnbook/input/titanic/t..., PartitionFilters: [], PushedFilters: [IsNotNull(PassengerId)], ReadSchema: struct<PassengerId:int,Pclass:int,Name:string,Sex:string,Age:double>


In [27]:
"""
Partitioning

Having too many partitions can cause inefficiencies. See: Top 5 Mistakes when writing Spark applications
For Spark, rule of thumb is to have around 128MB per partition.
"""

df1.rdd.getNumPartitions()
#df.repartition(numPartitions)
#df.coalaesce(numPartitions)

1

It seems like in this version of Spark, the automatic partitioning is more accurate than when the guide on https://changhsinlee.com/pyspark-dataframe-basics/ was written for Spark 2.1

If there are too many/too few partitions, we can repartition using df.repartition(numPartitions)
If there are close to 2000 partitions, it's recommended to bump that number up to slightly higher than 2000, because Spark uses a different, more compressed/efficient data structure for bookkeeping during block shuffling when there are more than 2000 partitions.

If we know for certain we're reducing the number of partitions, we can use coalesce() instead of repartition to avoid a full reshuffle, by retaining the upcoming number of partitions and distributing  the data from the extra partitions onto them.

In [34]:
#Supposedly gets the number of nodes in the cluster, including head and worker nodes
sc._jsc.sc().getExecutorMemoryStatus().keySet().size()

1